In [ ]:
#• Connect to Snowflake, extract some sample data, and perform feature engineering.  

from snowflake.snowpark import Session

# Connection parameters
connection_parameters = {
    "account": "XQQJITZ-WA31173",
    "user": "xyz",
    "password": "xyz@12345",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "SNOWFLAKE_SAMPLE_DATA",
    "schema": "TPCH_SF100"
}

# Create session
session = Session.builder.configs(connection_parameters).create()
print("✅ Connection Successful!")

In [ ]:
tables = session.sql("SHOW TABLES").collect()
for t in tables:
    print(t['name'])


In [ ]:
tables = session.sql("SHOW TABLES").collect()
print("Available tables in TPCH_SF100:")
for t in tables:
    print(t['name'])



In [ ]:
# Extract Sample Data
customer_df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF100.CUSTOMER")
customer_df.show(10)


In [ ]:
#Perform Feature Engineering
from snowflake.snowpark.functions import col, length, when

# Create features
features_df = customer_df.select(
    col("C_CUSTKEY"),
    col("C_ACCTBAL"),
    length(col("C_NAME")).alias("CUST_NAME_LENGTH"),
    when(col("C_ACCTBAL") > 5000, 1).otherwise(0).alias("HIGH_ACCT_BAL")
)

# Show features
features_df.show(10)


In [ ]:
# Q • Store the features in a Feature Store (e.g., Snowflake FS, Databricks FS, or AWS Feature Store).  


In [ ]:
from snowflake.snowpark.functions import col, length, when

# Example feature engineering
features_df = customer_df.select(
    col("C_CUSTKEY"),
    col("C_ACCTBAL"),
    length(col("C_NAME")).alias("CUST_NAME_LENGTH"),
    when(col("C_ACCTBAL") > 5000, 1).otherwise(0).alias("HIGH_ACCT_BAL")
)


In [ ]:
# Create a table to store features
session.sql("""
CREATE OR REPLACE TABLE CUSTOMER_FEATURE_STORE (
    C_CUSTKEY NUMBER,
    C_ACCTBAL NUMBER(12,2),
    CUST_NAME_LENGTH NUMBER,
    HIGH_ACCT_BAL NUMBER
)
""").collect()
print("✅ Feature Store table created!")


In [ ]:
# Write features into the Feature Store table
features_df.write.save_as_table("CUSTOMER_FEATURE_STORE", mode="overwrite")
print("✅ Features stored in Snowflake Feature Store!")


In [ ]:
# Load features from Feature Store
fs_features_df = session.table("CUSTOMER_FEATURE_STORE")
fs_features_df.show(10)


In [ ]:
# Retrieve features and use them for model training.

In [ ]:
# Load features from Snowflake
features_df = session.table("CUSTOMER_FEATURE_STORE")

# Show first 10 rows
features_df.show(10)

# Convert to Pandas for ML
features_pd = features_df.to_pandas()
print(features_pd.head())


In [ ]:
from sklearn.model_selection import train_test_split

# Features (X) and target (y)
X = features_pd[['C_ACCTBAL', 'CUST_NAME_LENGTH']]
y = features_pd['HIGH_ACCT_BAL']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
